# Competição ML @SBS/DAA - 5ª Edição (2022/2023)

Esta competição está relacionada com o Trabalho Prático de Grupo das UCs do perfil Machine Learning: Fundamentos e Aplicações da Uminho e da UC Dados e Aprendizagem Automática, tendo, como destinatários, alunos do Mestrado em Engenharia Informática, do Mestrado em Matemática e Computação, e do Mestrado em Engenharia de Sistemas.

# Descrição da Competição
A previsão da quantidade de incidentes rodoviários é um conhecido problema de características estocásticas, não-lineares. Tem, contudo, aparecido na literatura um conjunto de modelos que demonstram um potencial assinalável neste tipo de previsões. Com isso em consideração, foi construído um dataset que contém dados referentes à quantidade e características dos incidentes rodoviários que ocorreram na cidade de Guimarães em 2021 (o dataset cobre um período que vai desde o dia 01 de Janeiro de 2021 até ao dia 31 de Dezembro do mesmo ano).

Com esta competição espera-se que os alunos desenvolvam e otimizem modelos de Machine Learning que sejam capazes de prever o número de incidentes rodoviários que irão acontecer na cidade de Guimarães a uma determinada hora.

## Imports utilizados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# matplotlib inline

### Mudar o modo que a Interactive shell imprime as variáveis

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Carregar o dataset de treino

In [ ]:
TRAINING_DATASET_SOURCE = 'training_data.csv'
TEST_DATASET_SOURCE = 'test_data.csv'

In [ ]:
train_df = pd.read_csv(TRAINING_DATASET_SOURCE)
test_df = pd.read_csv(TEST_DATASET_SOURCE)

## Seed utilizada

In [ ]:
SEED = 101

## Exploração de dados

In [ ]:
train_df.head()

train_df.describe()

train_df.info()

In [ ]:
for column in train_df.columns:
    train_df[column].value_counts()

## Visualização gráfica dos dados

In [ ]:
sns.heatmap(train_df.corr(numeric_only=True))

In [ ]:
sns.heatmap(train_df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
sns.histplot(train_df['luminosity'])

In [ ]:
sns.histplot(train_df['avg_rain'])

In [ ]:
sns.displot(train_df['incidents'], kde=True)

In [ ]:
sns.displot(train_df['avg_wind_speed'], kde=True)

In [ ]:
train_df.isna().sum()

## Feature Importance

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# define the model
model = LinearRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

## Preparação de Dados

In [ ]:
from pandas import DataFrame

dropped_columns = ['city_name', 'magnitude_of_delay','avg_precipitation']

# noinspection PyPep8Naming
def decision_tree_data_preparation(df: DataFrame) -> DataFrame:

    prep_df = df.drop(dropped_columns, axis=1)

    ### Agrupar em 2 bins a feature 'avg_rain'
    prep_df.loc[prep_df['avg_rain'] == 'Sem Chuva', 'rain_level'] = 0
    prep_df.loc[prep_df['avg_rain'] != 'Sem Chuva', 'rain_level'] = 1

    prep_df.drop(columns=['avg_rain'], inplace=True)

    ### Agrupar em 3 bins a feature 'luminosity'
    prep_df.loc[prep_df['luminosity'] == 'LOW_LIGHT', 'luminosity_binned'] = 0
    prep_df.loc[prep_df['luminosity'] == 'LIGHT', 'luminosity_binned'] = 1
    prep_df.loc[prep_df['luminosity'] == 'DARK', 'luminosity_binned'] = 2

    prep_df.drop(columns=['luminosity'], inplace=True)
    
    ### Extrair a hora e dia da semana da feature 'record_date'
    record_date = pd.DatetimeIndex(prep_df['record_date'])

    prep_df['record_date_hour'] = record_date.hour
    prep_df['record_date_day'] = record_date.day
    prep_df['record_date_month'] = record_date.month
    prep_df['record_date_weekday'] = record_date.weekday
    
    prep_df.drop(columns=['record_date'], inplace=True)

    ### Quantificar a feature 'affected_roads' para o número único de estradas afetadas
    road_quantity = []
    for line in prep_df['affected_roads']:
        unique_roads = set(str(line).split(','))
        valid_roads = [elem for elem in unique_roads if elem != '']
        count = len(valid_roads)
        road_quantity.append(count)

    prep_df['affected_roads'] = road_quantity

    ### Converter a feature 'delay_in_seconds' para 'delay_in_hours' de modo a reduzir o intervalo de valores
    delay_in_hours = []
    for seconds in prep_df['delay_in_seconds']:
        hours = seconds / 3600
        delay_in_hours.append(hours)

    prep_df['delay_in_hours'] = delay_in_hours
    prep_df.drop(columns=['delay_in_seconds'], inplace=True)

    return prep_df

#prep = decision_tree_data_preparation(train_df)
#sns.heatmap(prep.corr(numeric_only=True))
#prep.describe()

### Divisão do dataset de treino em dados de treino e de teste

In [ ]:
from sklearn.model_selection import train_test_split

X = decision_tree_data_preparation(train_df)

y = X['incidents']

X.drop(columns=['incidents'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

### Modelo de Árvore de decisão

In [ ]:
clf = DecisionTreeClassifier(random_state=SEED)

clf.fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=5)

# Validação cruzada (K Fold)
from sklearn.svm import SVC

cross_valid_model = SVC(random_state=2021)
scores = cross_val_score(cross_valid_model, X, np.ravel(y), cv=2)

scores

print("Accuracy de %0.2f com um desvio padrão de %0.2f" % (scores.mean(), scores.std()))

#### Obter as previsões

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)
# TP FP
# FN TN
disp = ConfusionMatrixDisplay(cm)

disp.plot()

In [ ]:
accuracy_score(y_test, predictions)
# (TP + TN) / (TP + FP + FN + TN)

precision_score(y_test, predictions, average='micro')
# TP / (TP + FP)

recall_score(y_test, predictions, average='micro')
# TP / (TP + FN)

# Reading the test dataset

In [ ]:
test_df.head()
test_df.info()
test_df.describe()

In [ ]:
X = decision_tree_data_preparation(test_df)

# Getting the predictions

In [ ]:
predictions = clf.predict(X)

predictions_df = pd.DataFrame(predictions)

predictions_df.index += 1

predictions_df.to_csv("submission.csv", header=['Incidents'], index_label='RowId')